In [12]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, ConcatDataset
import pandas as pd
import os
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

data_dir = "/home/23ucs747/mini-project-sem5/dataset_split"
model_path = "/home/23ucs747/mini-project-sem5/models/efficientnet_b0_base.pth"
output_csv_all = "/home/23ucs747/mini-project-sem5/predictions_corn_v1.csv"
output_csv_low = "/home/23ucs747/mini-project-sem5/selected_low_confidence.csv"


Using device: cuda


In [13]:
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [14]:
train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=test_transforms)
test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=test_transforms)

combined_dataset = ConcatDataset([train_dataset, test_dataset])
data_loader = DataLoader(combined_dataset, batch_size=16, shuffle=False, num_workers=4)

class_names = train_dataset.classes
print(f"Loaded {len(train_dataset) + len(test_dataset)} total images across {len(class_names)} classes.")

Loaded 4188 total images across 4 classes.


In [15]:
from torch import nn
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 4)  # 4 classes
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
print("EfficientNet-B0 model loaded successfully.")

EfficientNet-B0 model loaded successfully.


In [16]:
results = []

# Function to get image path from ConcatDataset
def get_image_path(idx):
    if idx < len(train_dataset):
        return train_dataset.samples[idx][0]
    else:
        return test_dataset.samples[idx - len(train_dataset)][0]

with torch.no_grad():
    for batch_idx, (images, labels) in enumerate(tqdm(data_loader, desc="Running inference")):
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)
        confidences, preds = torch.max(probs, 1)

        for i in range(len(images)):
            global_idx = batch_idx * data_loader.batch_size + i
            file_path = get_image_path(global_idx)
            true_label = class_names[labels[i].item()]
            pred_label = class_names[preds[i].item()]
            conf = confidences[i].item()
            results.append({
                "image_path": file_path,
                "true_label": true_label,
                "pred_label": pred_label,
                "confidence": conf
            })

Running inference: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 262/262 [00:10<00:00, 25.33it/s]


In [17]:
df = pd.DataFrame(results)
df.to_csv(output_csv_all, index=False)
print(f"All predictions saved to: {output_csv_all}")


threshold = 0.8
low_conf_df = df[df["confidence"] < threshold]
low_conf_df.to_csv(output_csv_low, index=False)

print(f"Selected {len(low_conf_df)} low-confidence samples (< {threshold})")
print(f"Low-confidence file saved to: {output_csv_low}")

All predictions saved to: /home/23ucs747/mini-project-sem5/predictions_corn_v1.csv
Selected 829 low-confidence samples (< 0.8)
Low-confidence file saved to: /home/23ucs747/mini-project-sem5/selected_low_confidence.csv
